In [1]:
from autogen.agentchat import ConversableAgent
import autogen

config_list = autogen.config_list_from_json(
    env_or_file=r"C:\Users\sheelaj.babu\Desktop\Autogen-Training\Exercises\OAI_CONFIG_LIST.json")

llm_config = {"config_list": config_list}

c:\Users\sheelaj.babu\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Creating the needed agents

In [2]:
onboarding_personal_information_agent = ConversableAgent(
    name="onboarding_personal_information_agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [3]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [4]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [5]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

### Creating tasks
Now, you can craft a series of tasks to facilitate the onboarding process.

In [6]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

#### Start the onboarding process

In [7]:
from autogen import initiate_chats
chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
onboarding_personal_information_agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


c:\Users\sheelaj.babu\AppData\Local\Programs\Python\Python310\lib\site-packages\autogen\agentchat\chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


customer_proxy_agent (to onboarding_personal_information_agent):

Sheelaj

--------------------------------------------------------------------------------
onboarding_personal_information_agent (to customer_proxy_agent):

Thank you for providing your name! Could you please also tell me your location?

--------------------------------------------------------------------------------
customer_proxy_agent (to onboarding_personal_information_agent):

Kanpur

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Topic preference Agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
{
  "name": "Sheelaj",
  "location": "Kanpur"
}

------------------------------------------------------------------------------

c:\Users\sheelaj.babu\AppData\Local\Programs\Python\Python310\lib\site-packages\autogen\agentchat\conversable_agent.py:1258: UserWarning: Cannot extract summary using reflection_with_llm: Error code: 400 - {'error': {'message': "'Onboarding Topic preference Agent' does not match '^[a-zA-Z0-9_-]{1,64}$' - 'messages.0.name'", 'type': 'invalid_request_error', 'param': None, 'code': None}}. Using an empty str as summary.
  warnings.warn(


Customer Engagement Agent (to customer_proxy_agent):

Hey Sheelaj from Kanpur! I'm here to help you find something fun to read. What kind of topics are you interested in?

--------------------------------------------------------------------------------


c:\Users\sheelaj.babu\AppData\Local\Programs\Python\Python310\lib\site-packages\autogen\agentchat\conversable_agent.py:1258: UserWarning: Cannot extract summary using reflection_with_llm: Error code: 400 - {'error': {'message': "'Customer Engagement Agent' does not match '^[a-zA-Z0-9_-]{1,64}$' - 'messages.1.name'", 'type': 'invalid_request_error', 'param': None, 'code': None}}. Using an empty str as summary.
  warnings.warn(
